In [1]:
import os
import cv2
import mmcv
import numpy as np
import matplotlib.pyplot as plt
from mmengine.config import Config
from mmengine.registry import init_default_scope
from mmdet.apis import inference_detector, init_detector
from mmrotate.visualization import RotLocalVisualizer

In [2]:
# Configuration and checkpoint files
config_file = '../configs/rhino/rhino_phc_haus-4scale_r50_2xb2-36e_brickkilns.py'
checkpoint_file = '../work_dirs/rhino_phc_haus-4scale_r50_2xb2-36e_brickkilns/epoch_36.pth'
image_dir = '../data/brickkilns/test/images'
out_dir = '../results/bihar/images'
ann_dir = '../results/bihar/annfiles'

In [3]:
# Set the GPU to use
os.environ['CUDA_VISIBLE_DEVICES'] = '0,1'

# Load the configuration file
cfg = Config.fromfile(config_file)
init_default_scope(cfg.get('default_scope', 'mmrotate'))

# Initialize the model
model = init_detector(cfg, checkpoint_file, device='cuda:0')

# Create output directories if they don't exist
os.makedirs(out_dir, exist_ok=True)
os.makedirs(ann_dir, exist_ok=True)

# Initialize the visualizer
visualizer = RotLocalVisualizer()

image_files = os.listdir(image_dir)
image_files = sorted(image_files)
num_images = len(image_files)

Loads checkpoint by local backend from path: ../work_dirs/rhino_phc_haus-4scale_r50_2xb2-36e_brickkilns/epoch_36.pth


In [4]:
for i, image_name in enumerate(image_files[:1]):
    print(f'Processing image {i + 1}/{num_images}...')
    print(image_name)
    image_path = os.path.join(image_dir, image_name)
    result = inference_detector(model, image_path)

    # Save the visualized test images
    visualizer.add_datasample(
        image_name,
        mmcv.imread(image_path),
        result,
        show=False,
        draw_gt=False,
        out_file=os.path.join(out_dir, image_name)
    )

    # Save annotations in DOTA format
    ann_file = os.path.join(ann_dir, f'{os.path.splitext(image_name)[0]}.txt')
    print(result)
    
    with open(ann_file, 'w') as f:
        bboxes = result.pred_instances.bboxes.cpu().numpy()
        scores = result.pred_instances.scores.cpu().numpy()
        labels = result.pred_instances.labels.cpu().numpy()
        print(len(bboxes), len(scores), len(labels))

        for bbox, score, label in zip(bboxes, scores, labels):
            if score < 0.3:  # Confidence threshold to filter low-confidence detections
                continue

            # Convert (cx, cy, w, h, angle) to DOTA format
            cx, cy, w, h, angle = bbox  # Rotated bbox format: (center_x, center_y, width, height, angle)
            points = cv2.boxPoints(((cx, cy), (w, h), np.degrees(angle)))  # Get 4 corner points
            points = points.flatten()
            print(points)
            # Write to DOTA format file
            f.write(f"{' '.join(map(str, points))} {label} {score:.2f}\n")

print('Done.')

Processing image 1/687...
9295968_2901852.png
01/30 23:56:27 - mmengine - WARNING - "FileClient" will be deprecated in future. Please use io functions in https://mmengine.readthedocs.io/en/latest/api/fileio.html#file-io
01/30 23:56:27 - mmengine - WARNING - "HardDiskBackend" is the alias of "LocalBackend" and the former will be deprecated in future.
<DetDataSample(

    META INFORMATION
    img_path: '../data/brickkilns/test/images/9295968_2901852.png'
    scale_factor: (1.0, 1.0)
    pad_shape: (640, 640)
    batch_input_shape: (640, 640)
    img_id: 0
    ori_shape: (640, 640)
    img_shape: (640, 640)

    DATA FIELDS
    pred_instances: <InstanceData(
        
            META INFORMATION
        
            DATA FIELDS
            bboxes: tensor([[ 63.1395,  19.7437,  22.0124,  10.3918,   1.7019],
                        [256.6378,  36.8707,  20.1259,   9.6163,   1.7552],
                        [ 63.1395,  19.7437,  22.0124,  10.3918,   1.7019],
                        ...,
    

/home/shardul.junagade/my-work/domain-adaptation-brick-kilns/RHINO/.venv/lib/python3.8/site-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  ../aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)
